Load packages

In [294]:
import sklearn
import pandas as pd
import numpy as np
from math import pi

import matplotlib.pyplot as plt
import seaborn as sns



In [302]:
columns = ['GAME_DATE_EST',
 'GAME_ID',
 'HOME_TEAM_WINS',
 'HOME_TEAM',
 'VISITOR_TEAM_WINS',
 'VISITOR_TEAM',
 'HOME_TEAM_WIN_LAST_10_GAMES',
 'VISITOR_TEAM_WIN_LAST_10_GAMES']
teams = pd.read_csv('../nba-games/teams.csv', usecols=['TEAM_ID', 'ABBREVIATION'])
teams.rename(columns={"ABBREVIATION":"NAME"}, inplace=True)
games = pd.read_csv("../nba-games/games.csv",parse_dates=["GAME_DATE_EST"],infer_datetime_format=True)
games = games.drop_duplicates()\
    .drop(columns=["TEAM_ID_home", "TEAM_ID_away", "GAME_STATUS_TEXT"], axis=1)

df = games[games.SEASON == 2018]
df["VISITOR_TEAM_WINS"] = df["HOME_TEAM_WINS"]==0
print(teams.shape)
print(games.shape)
print(df.shape)

(30, 2)
(23195, 18)
(1378, 19)


/home/nathaniel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [303]:
def add_team_abbr_column(is_home):
    global df
    if is_home:
        df = pd.merge(df, teams, left_on='HOME_TEAM_ID', right_on='TEAM_ID')
        df.rename(columns={"NAME":"HOME_TEAM"}, inplace=True)
        df = df.drop(columns=["HOME_TEAM_ID"], axis=1)
    else:
        df = pd.merge(df, teams, left_on='VISITOR_TEAM_ID', right_on='TEAM_ID')
        df.rename(columns={"NAME":"VISITOR_TEAM"}, inplace=True)
        df = df.drop(columns=["VISITOR_TEAM_ID"], axis=1)
    df = df.drop(columns=["TEAM_ID"], axis=1)


add_team_abbr_column(True)
add_team_abbr_column( False)
df = df.sort_values("GAME_DATE_EST", ascending=False).reset_index(drop=True)
df.head(5)

,GAME_DATE_EST,GAME_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS,HOME_TEAM,VISITOR_TEAM
0,2019-06-13,41800406,2018,110.0,0.488,0.700,0.355,28.0,42.0,114.0,0.476,0.793,0.394,25.0,39.0,0,True,GSW,TOR
1,2019-06-10,41800405,2018,105.0,0.447,0.778,0.250,19.0,43.0,106.0,0.463,0.714,0.476,27.0,37.0,0,True,TOR,GSW
2,2019-06-07,41800404,2018,92.0,0.449,0.667,0.296,26.0,42.0,105.0,0.419,0.958,0.313,22.0,39.0,0,True,GSW,TOR
3,2019-06-05,41800403,2018,109.0,0.396,0.833,0.333,25.0,41.0,123.0,0.524,0.952,0.447,30.0,40.0,0,True,GSW,TOR
4,2019-06-02,41800402,2018,104.0,0.372,0.885,0.289,17.0,49.0,109.0,0.463,0.870,0.382,34.0,42.0,0,True,TOR,GSW


In [304]:
df_preprocessed = pd.DataFrame()

df_preprocessed = df[["HOME_TEAM", "VISITOR_TEAM"]].copy()

df_preprocessed.sample()

,HOME_TEAM,VISITOR_TEAM
973,DAL,LAC


In [305]:
def last_10_games(team, game_date):
    query = (game_date > df.GAME_DATE_EST) & \
            ((df.HOME_TEAM == team) | \
            (df.VISITOR_TEAM == team))
    team_df = df.loc[query].head(10)
    balance = 0
    for idx, row in team_df.iterrows():
        if row.HOME_TEAM == team:
            balance += row.HOME_TEAM_WINS
        else:
            balance += row.VISITOR_TEAM_WINS
    return balance

In [306]:
#df_preprocessed["HOME_TEAM_WIN_LAST_10_GAMES"] = 0
#df_preprocessed["VISITOR_TEAM_WIN_LAST_10_GAMES"] = 0
for idx, row in df.iterrows():
   df.at[idx, "HOME_TEAM_WIN_LAST_10_GAMES"] = last_10_games(row["HOME_TEAM"], row["GAME_DATE_EST"])
   df.at[idx, "VISITOR_TEAM_WIN_LAST_10_GAMES"] = last_10_games(row["VISITOR_TEAM"], row["GAME_DATE_EST"])


In [307]:
test = df[columns]
query = ('2019-06-10' > df.GAME_DATE_EST) & \
            ((df.HOME_TEAM == "TOR") | \
            (df.VISITOR_TEAM == "TOR"))
test[query].head(20)

,GAME_DATE_EST,GAME_ID,HOME_TEAM_WINS,HOME_TEAM,VISITOR_TEAM_WINS,VISITOR_TEAM,HOME_TEAM_WIN_LAST_10_GAMES,VISITOR_TEAM_WIN_LAST_10_GAMES
2,2019-06-07,41800404,0,GSW,True,TOR,7.0,7.0
3,2019-06-05,41800403,0,GSW,True,TOR,7.0,6.0
4,2019-06-02,41800402,0,TOR,True,GSW,7.0,7.0
5,2019-05-30,41800401,1,TOR,False,GSW,7.0,8.0
6,2019-05-25,41800306,1,TOR,False,MIL,6.0,6.0
7,2019-05-23,41800305,0,MIL,True,TOR,7.0,5.0
8,2019-05-21,41800304,1,TOR,False,MIL,5.0,8.0
10,2019-05-19,41800303,1,TOR,False,MIL,5.0,9.0
12,2019-05-17,41800302,1,MIL,False,TOR,9.0,6.0
14,2019-05-15,41800301,1,MIL,False,TOR,8.0,7.0


In [308]:
#print(df_preprocessed.shape)
#print(df.shape)
#
#df[["VISITOR_TEAM_WIN_LAST_10_GAMES", "HOME_TEAM_WIN_LAST_10_GAMES"]]


df_preprocessed = pd.concat(
    [df_preprocessed,
     df[["HOME_TEAM_WIN_LAST_10_GAMES", "VISITOR_TEAM_WIN_LAST_10_GAMES"]]]
    , axis=1
)
df_preprocessed.head(5)





,HOME_TEAM,VISITOR_TEAM,HOME_TEAM_WIN_LAST_10_GAMES,VISITOR_TEAM_WIN_LAST_10_GAMES
0,GSW,TOR,7.0,7.0
1,TOR,GSW,7.0,7.0
2,GSW,TOR,7.0,7.0
3,GSW,TOR,7.0,6.0
4,TOR,GSW,7.0,7.0
